In [5]:
from openai import OpenAI
import os
client = OpenAI()

response = client.chat.completions.create(
  model=os.getenv("GPT_MODEL"),
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "last 5 fifa worldcup winner?"}
  ]
)
print(response.choices[0].message.content)

{
    "winners": [
        "2018 - France",
        "2014 - Germany",
        "2010 - Spain",
        "2006 - Italy",
        "2002 - Brazil"
    ]
}


In [22]:
from openai import OpenAI
import json

client = OpenAI()

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What date is today?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    print(response_message)
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
output = run_conversation()


ChatCompletionMessage(content="Today's date is October 21, 2023.", role='assistant', function_call=None, tool_calls=None)


In [21]:
output.choices[0].message.content

AttributeError: 'NoneType' object has no attribute 'choices'

In [4]:
import requests
def get_population():
        """get usa population"""
        url = "https://datausa.io/api/data?drilldowns=Nation&measures=Population"
        response = requests.get(url )
    
        if response.status_code == 200:
            print("Success:", response.json())
        else:
            print("Error:", response.status_code, response.text)
print(get_population())

Success: {'data': [{'ID Nation': '01000US', 'Nation': 'United States', 'ID Year': 2022, 'Year': '2022', 'Population': 331097593, 'Slug Nation': 'united-states'}, {'ID Nation': '01000US', 'Nation': 'United States', 'ID Year': 2021, 'Year': '2021', 'Population': 329725481, 'Slug Nation': 'united-states'}, {'ID Nation': '01000US', 'Nation': 'United States', 'ID Year': 2020, 'Year': '2020', 'Population': 326569308, 'Slug Nation': 'united-states'}, {'ID Nation': '01000US', 'Nation': 'United States', 'ID Year': 2019, 'Year': '2019', 'Population': 324697795, 'Slug Nation': 'united-states'}, {'ID Nation': '01000US', 'Nation': 'United States', 'ID Year': 2018, 'Year': '2018', 'Population': 322903030, 'Slug Nation': 'united-states'}, {'ID Nation': '01000US', 'Nation': 'United States', 'ID Year': 2017, 'Year': '2017', 'Population': 321004407, 'Slug Nation': 'united-states'}, {'ID Nation': '01000US', 'Nation': 'United States', 'ID Year': 2016, 'Year': '2016', 'Population': 318558162, 'Slug Nation'

In [8]:
import requests
from openai import OpenAI

# Fetch data from the API
def fetch_data():
    url = "https://datausa.io/api/data?drilldowns=Nation&measures=Population"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None
client  = OpenAI()
# Generate a response using OpenAI's chat completion
def generate_response(data):
    
    # Construct the message with fetched data
    message = f"The latest population data for the USA is as follows:\n{data}"
    
    # Create a chat completion request
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",  # or the model you are using
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Can you provide the population data for the USA in 2019?"},
            {"role": "assistant", "content": message}
        ]
    )
    
    # Extract and return the assistant's reply
    return response.choices[0].message.content

# Main function to fetch data and generate response
def main():
    data = fetch_data()
    if data:
        response = generate_response(data)
        print(response)
    else:
        print("Failed to fetch data from the API.")

if __name__ == "__main__":
    main()


The population of the USA in 2019 was approximately 324,697,795.


In [13]:
def get_cat_fact():
    url = "https://vapt-mp.myxenius.com/thirdparty/api/login?login_id=2121901&password=2121901"
    response = requests.get(url )

    if response.status_code == 200:
        # print("Success:", response.json())
        print("cat fact data success")

        return response.json()
    else:
        print("Error in cat fact:", response.status_code, response.text)
print(get_cat_fact())

cat fact data success
{'rc': 0, 'message': 'SUCCESS', 'application': 'AMR', 'welcome_message': "Now, track your electricity consumption on your phone! Xenius SEMS app allows you to monitor your consumption, recharge your prepaid meters online and also compare your electricity bills over time.What's more? This version of the app opens up possibilities of viewing your sanctioned load, setting notifications for Source Change, Low Balance, Power Cut / Restore and more.Facing any issues? Get connected with the support team through your Settings. We hope our app helps you track your electricity usage and optimise accordingly.", 'app_base_url': 'www.myxenius.com', 'payment_trans': 'If the recharge is conducted using a Debit/Commercial/Corporate Card, MDR charges will be subtracted from the recharge amount', 'resource': {'android_version': '15', 'ios_version': '1', 'mobile_app_status': 'Y', 'location_id': '2121901', 'tp_token_id': '212190142a', 'ism_token': None, 'login_count': '226', 'device_